# Prey notebook

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import neat
import matplotlib.pyplot as plt

import pickle
import multimodal_mazes
from tqdm import tqdm

## Ideas 

* Odour - constant but noisy. 
* Sound - reliable but infrequent.
* Rather than resetting the env to zero every step, you could decay it. E.g. env[:,:,:-1] *= 0.8 + blur. 
* Analysis: n_prey caught, speed, costs (e.g. movement vs food). 
* Evolve prey against different algorithms. Then, evolve predators against these prey.   

## Rule-based Agents

In [ ]:
# Hyperparameters 
size = 7
n_prey = 10
n_steps = 50
n_trials = 100
pk = 5 # the width of the prey's Gaussian signal (in rc)
scenario = "Hunting"
pm = 0.1
pe = 0.1

### Fitness vs noise

In [ ]:
# Fitness vs noise
noises = np.linspace(start=0.0, stop=2.0, num=13)
policies = multimodal_mazes.AgentRuleBased.policies + multimodal_mazes.AgentRuleBasedMemory.policies
colors = multimodal_mazes.AgentRuleBased.colors + multimodal_mazes.AgentRuleBasedMemory.colors
results = np.zeros((len(noises), len(policies)))

# Test agents
for a, noise in enumerate(tqdm(noises)):

    for b, policy in enumerate(policies): 
        if policy in multimodal_mazes.AgentRuleBased.policies:
            agnt = multimodal_mazes.AgentRuleBased(location=None, channels=[1,1], policy=policy)
        else:
            agnt = multimodal_mazes.AgentRuleBasedMemory(location=None, channels=[1,1], policy=policy)
            agnt.alpha = 0.6

        fitness, _, _, _ = multimodal_mazes.eval_predator_fitness(n_trials=n_trials, size=size, agnt=agnt, sensor_noise_scale=noise, n_prey=n_prey, pk=pk, n_steps=n_steps, scenario=scenario, pm=pm, pe=pe)

        results[a, b] = fitness

for b, policy in enumerate(policies): 
    plt.plot(noises, results[:,b], color=colors[b], label=policy)

plt.ylim([0, 1.05])
plt.ylabel('Fitness')
plt.xlabel('Sensor Noise')
plt.legend()

In [ ]:
# Fitness vs noise AUC 
auc = np.trapz(y=results.T, x=noises, axis=1)
for b, _ in enumerate(policies): 
    ml, sl, _ = plt.stem(b, auc[b])
    ml.set_color(colors[b])
    sl.set_color(colors[b])
plt.xticks(range(len(policies)), policies, rotation='vertical')
plt.ylabel('AUC');

### Finding alpha for the memory based agents

In [ ]:
# Fitness vs noise
noises = np.linspace(start=0.0, stop=2.0, num=13)
policies = multimodal_mazes.AgentRuleBasedMemory.policies
alphas = np.linspace(start=0.0, stop=2.0, num=11)
results = np.zeros((len(noises), len(policies), len(alphas)))

# Test agents
for a, noise in enumerate(tqdm(noises)):

    for b, policy in enumerate(policies): 

        for c, alpha in enumerate(alphas):
            agnt = multimodal_mazes.AgentRuleBasedMemory(location=None, channels=[1,1], policy=policy)
            agnt.alpha=alpha
            fitness, _, _, _ = multimodal_mazes.eval_predator_fitness(n_trials=n_trials, size=size, agnt=agnt, sensor_noise_scale=noise, n_prey=n_prey, pk=pk, n_steps=n_steps, scenario=scenario, pm=pm, pe=pe)
            results[a, b, c] = fitness

In [ ]:
# Fitness vs noise
colors = multimodal_mazes.AgentRuleBasedMemory.colors
auc = np.trapz(y=results.T, x=noises, axis=2)
for b, policy in enumerate(policies): 
    plt.plot(alphas, auc[:,b], color=colors[b], label=policy)

plt.ylabel('AUC')
plt.xlabel(r"$\alpha$")
plt.legend()

### Exploring task parameters 

In [ ]:
# Fitness vs noise
noises = np.linspace(start=0.0, stop=2.0, num=10)
policies = multimodal_mazes.AgentRuleBased.policies + multimodal_mazes.AgentRuleBasedMemory.policies
colors = multimodal_mazes.AgentRuleBased.colors + multimodal_mazes.AgentRuleBasedMemory.colors
pms = np.linspace(start=0.0, stop=1.0, num=10)
pes = np.linspace(start=0.0, stop=1.0, num=10)

# results = np.zeros((len(noises), len(policies), len(pms), len(pes)))

# # Test agents
# for a, noise in enumerate(tqdm(noises)):

#     for b, policy in enumerate(policies): 
#         if policy in multimodal_mazes.AgentRuleBased.policies:
#             agnt = multimodal_mazes.AgentRuleBased(location=None, channels=[1,1], policy=policy)
#         else:
#             agnt = multimodal_mazes.AgentRuleBasedMemory(location=None, channels=[1,1], policy=policy)
#             agnt.alpha = 0.6

#         for c, pm in enumerate(pms):
#             for d, pe in enumerate(pes):
#                 fitness, _, _, _ = multimodal_mazes.eval_predator_fitness(n_trials=n_trials, size=size, agnt=agnt, sensor_noise_scale=noise, n_prey=n_prey, pk=pk, n_steps=n_steps, scenario=scenario, pm=pm, pe=pe)

#                 results[a, b, c, d] = fitness

# np.save('results_noise', results) 

In [ ]:
# Load results 
results = np.load("../results/test17/results_noise.npy")
results.shape

In [ ]:
# Mean AUC 
auc = np.zeros((len(policies), len(pms), len(pes)))
for c, _ in enumerate(pms):
    for d, _ in enumerate(pes):
        auc[:,c,d] = np.trapz(y=results[:,:,c,d].T, x=noises, axis=1)

for b, _ in enumerate(policies): 
    ml, sl, _ = plt.stem(b, np.mean(auc[b]) - np.mean(auc[0]))
    ml.set_color(colors[b])
    sl.set_color(colors[b])
plt.xticks(range(len(policies)), policies, rotation='vertical')
plt.ylabel('Normalised mean AUC');

In [ ]:
results_diff = results[:,-1,:,:] - results[:,-4,:,:]
print(np.argwhere(results_diff == np.max(results_diff))) # noises, pms, pes 
print(results_diff.min(), results_diff.max())

In [ ]:
results_list = []
for a, noise in enumerate(noises):
    for b, pm in enumerate(pms):
        for c, pe in enumerate(pes):
            results_list.append([noise, pm, pe, results_diff[a,b,c]])
results_list = np.array(results_list)

In [ ]:
from sklearn.linear_model import LinearRegression
a = 0
plt.scatter(results_list[:,a], results_list[:,-1], c='k', marker='.', alpha=0.2)
model = LinearRegression().fit(results_list[:, a][:, None], results_list[:,-1])
plt.plot(
        results_list[:, a],
        model.predict(results_list[:, a][:, None]),
        color="xkcd:coral pink",
        alpha=1.0,
    )

## Plotting

In [ ]:
agnt = multimodal_mazes.AgentRuleBased(location=None, channels=[1,1], policy='Linear fusion')
noise = 0.1
# agnt = multimodal_mazes.AgentRuleBasedMemory(location=None, channels=[1,1], policy='Recurrent outputs')
# agnt.alpha=0.9
time, path, prey_state, preys = multimodal_mazes.predator_trial(size=size, agnt=agnt, sensor_noise_scale=noise, n_prey=n_prey, pk=pk,n_steps=n_steps, scenario=scenario, pm=pm, pe=pe)
print(prey_state)

In [ ]:
# Plotting
from matplotlib import colors
prey_markers = ['P', 'X']

# Environment 
pk_hw = pk // 2  # half width of prey's Gaussian signal (in rc)
env = np.zeros((size, size, len(agnt.channels) + 1))
env[:, :, -1] = 1.0
env = np.pad(env, pad_width=((pk_hw, pk_hw), (pk_hw, pk_hw), (0, 0)))
plt.imshow(1 - env[:, :, -1], cmap="binary", alpha=0.25)

# Path
cmap = colors.LinearSegmentedColormap.from_list(
    "", ["xkcd:teal blue", "xkcd:off white", "xkcd:coral"], N=n_steps
)
for t in range(len(path) - 1):
    plt.plot([path[t, 1], path[t + 1, 1]], [path[t, 0], path[t + 1, 0]], c=cmap(t), zorder=0)
    plt.scatter(path[t + 1, 1], path[t + 1, 0], s=30, color=cmap(t), zorder=1)

# Prey 
for prey in preys:
    path = np.array(prey.path)
    if scenario == "Foraging":
        plt.scatter(path[0,1], path[0,0], color='k', alpha=0.5, marker=prey_markers[prey.cues], zorder=2)

    if scenario == "Hunting":
        plt.scatter(path[-1,1], path[-1,0], color='k', alpha=0.5, marker=prey_markers[0], zorder=2)

# Adjust axes 
plt.xlim([(pk//2) - 1, size + pk//2])
plt.ylim([size + pk//2, (pk//2) - 1]) 
plt.axis("off")


### WIP: Video

In [ ]:
cmap = colors.LinearSegmentedColormap.from_list(
    "", ["white", "xkcd:ultramarine"]
)

cmap1 = colors.LinearSegmentedColormap.from_list(
    "", ["white", "xkcd:magenta"]
)

plt.imshow((cmap(env[:,:,0]) + cmap1(env[:,:,1]))/2)
plt.imshow(1 - env[:, :, -1], cmap="binary", alpha=0.25)
# Adjust axes 
plt.xlim([(pk//2) - 1, size + pk//2])
plt.ylim([size + pk//2, (pk//2) - 1]) 
plt.axis("off")